In [1]:
import requests
import dotenv
import os
import pandas as pd
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
import json
# Load environment variables from .env file
dotenv.load_dotenv()

True

In [2]:
#read OURA_PAT from environment variables
OURA_PAT = os.getenv("OURA_PAT")

In [ ]:
import requests 
url = 'https://api.ouraring.com/v2/usercollection/heartrate' 

headers = { 
  'Authorization': f'Bearer {OURA_PAT}' 
}

start_year = 2021
end_year = 2025

current_start_date = date(start_year, 11, 1)
overall_end_date = date(end_year, 7, 25)

pbar = tqdm(total=(overall_end_date - current_start_date).days // 30 + 1, desc="Fetching Heart Rate Data")

all_data = []

print("starting to fetch heart rate data...")
while current_start_date < overall_end_date:
    current_end_date = current_start_date + relativedelta(months=1) - timedelta(days=1)
    
    if current_end_date > overall_end_date:
        current_end_date = overall_end_date
    params = {
        'start_datetime': current_start_date.strftime('%Y-%m-%dT%H:%M:%S-08:00'),
        'end_datetime': current_end_date.strftime('%Y-%m-%dT%H:%M:%S-08:00')
    }

    response = requests.request('GET', url, headers=headers, params=params)
    response_json = response.json()
    pbar.update(1)

    #update current_start_date to the next month
    

    if 'data' in response_json:
      #save data to json
      file_name = f"data/heartrate_{current_start_date.strftime('%Y-%m-%d')}_to_{current_end_date.strftime('%Y-%m-%d')}.json"
      with open(file_name, 'w') as f:
          json.dump(response_json['data'], f, indent=4)
      
      all_data.extend(response_json['data'])
    else:
      print(f"No data for period: {current_start_date} to {current_end_date}")

    current_start_date = current_end_date + timedelta(days=1)

Fetching Heart Rate Data:  76%|███████▌  | 35/46 [02:46<00:52,  4.74s/it]


starting to fetch heart rate data...


KeyboardInterrupt: 

In [ ]:
#convert to pandas dataframe
heart_rate_df = pd.DataFrame(all_heart_rate_data)

KeyError: 'data'

In [ ]:
heart_rate_df 

,bpm,source,timestamp
0,73,awake,2021-11-17T16:46:57+00:00
1,64,live,2021-11-17T16:47:07+00:00
2,62,awake,2021-11-17T16:47:24+00:00
3,63,awake,2021-11-17T16:47:26+00:00
4,61,live,2021-11-17T16:47:38+00:00
...,...,...,...
2574,61,awake,2021-11-30T22:03:53+00:00
2575,54,awake,2021-11-30T22:04:02+00:00
2576,74,awake,2021-12-01T07:40:40+00:00
2577,85,awake,2021-12-01T07:40:43+00:00
